In [144]:
import pandas as pd
import numpy as np
import json

Grab all of the identifying information for the head of the household and the details of their first migration to the US

In [145]:
pers = pd.read_csv("./pers174.csv")
pers = pers[pers['relhead'] == 1]
pers = pers[~pers['statebrn'].isin([0])] # Filter for crossings with a known state of crossing
# pers = pers[~pers['placebrn'].isin([0])] # Filter for crossings with a known state of crossing
print(pers.shape)
pers_vars = ["COMMUN", "HHNUM", "PLACEBRN", "STATEBRN", "DOSTATEL", "DOPLACEL", "USYR1", "USDUR1", "USDOC1", "USSTATE1", "USPLACE1", "USMAR1", "USOCC1", "USWAGE1", "USBY1"]
pers = pers[[i.lower() for i in pers_vars]]
pers = pers[pers["statebrn"] != 8888]
pers = pers[pers["statebrn"] != 9999]
pers["placebrn"] = pers["placebrn"].astype(str).str.replace("8888", "0").astype(str)
pers["placebrn"] = pers["placebrn"].astype(str).str.replace("9999", "0").astype(str)
pers.head()

/opt/conda/envs/rapids/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (19,20,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,49,50,58,59,66,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(28842, 132)


,commun,hhnum,placebrn,statebrn,dostatel,doplacel,usyr1,usdur1,usdoc1,usstate1,usplace1,usmar1,usocc1,uswage1,usby1
0,1,1,0,11,8888,8888,8888,8888,8888,8888,8888,,8888,8888.0,8888
6,1,2,74,14,8888,8888,8888,8888,8888,8888,8888,,8888,8888.0,8888
15,1,3,0,11,14,72,1977,12,9999,105,7320,,529,9999.0,9999
25,1,4,0,11,8888,8888,8888,8888,8888,8888,8888,,8888,8888.0,8888
32,1,5,26,11,9,401,8888,8888,8888,8888,8888,,8888,8888.0,8888


Grab all of the details of the head of the household about their first MX-US border crossing

In [146]:
mig = pd.read_csv("./mig174.csv")
mig_vars = ["COMMUN", "HHNUM", "CRSYR1", "CRSST1", "CRSPL1", "CRSHOW1", "CRSCOY1", "CRSCST1", "CRSPAY1", "CRSDEP1", "CRSYES1"]
mig = mig[[i.lower() for i in mig_vars]]
mig = mig[~mig['crsst1'].isin([8888, 9999, 1111])] # Filter for crossings with a known state of crossing
mig = mig[~mig['crspl1'].isin([8888, 9999, 1111])] # Filter for crossings with a known place of crossing
print(mig.shape)
mig.head()

(5327, 11)


/opt/conda/envs/rapids/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (14,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,40,43,49,52,60,61,62,63,64,65,66,340,341,342,343,346,347,349,350,351,352,355,356,359,360,361,362,365,366,369,370,371,372,375,376,379,380,381,382,385,386,389,390,391,392,395,396,399,400,401,402,405,406,409,410,411,412,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,485,486,487,488,491,492,493,494,499,502,503,505,506,507,508,516,517,518,519,520,521,522,523,524,525,526,527,528,530,531,532,552,553,554,555,559,560,561) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,commun,hhnum,crsyr1,crsst1,crspl1,crshow1,crscoy1,crscst1,crspay1,crsdep1,crsyes1
3,1,14,1972,2,4,1,1,9999.0,9999,1,1
6,1,22,1980,2,4,1,1,150.0,9999,0,1
8,1,26,1978,2,4,1,1,75.0,9999,1,1
13,1,47,1980,2,4,2,2,8888.0,8888,1,1
19,1,67,1975,28,27,3,2,8888.0,8888,0,1


Variables to merge on

In [147]:
common_vars = ["COUNTRY", "COMMUN", "SURVEYPL", "SURVEYYR", "HHNUM"]

with open("./var_maps/state_to_place_map.json", "r") as f:
    state_to_place_map = json.load(f)
    
with open("./var_maps/us_places.json", "r") as f:
    us_places = json.load(f)

In [148]:
# pers_mig = pd.merge(pers, mig, on = [i.lower() for i in common_vars])

pers['ID'] = pers['commun'].astype(str) + "-" + pers['hhnum'].astype(str)
mig['ID'] = mig['commun'].astype(str) + "-" + mig['hhnum'].astype(str)

pers = pers.drop(["commun", "hhnum"], axis = 1)
mig = mig.drop(["commun", "hhnum"], axis = 1)

pers_mig = pd.merge(pers, mig, on = "ID")

pers_mig['cross_loc'] = pers_mig['crsst1'].astype(str) + "-" + pers_mig['crspl1'].astype(str)
pers_mig['cross_loc'] = pers_mig['cross_loc'].astype(str).map(state_to_place_map)
pers_mig['born_loc'] = pers_mig['statebrn'].astype(str) + "-" + pers_mig['placebrn'].astype(str)
pers_mig['born_loc'] = pers_mig['born_loc'].astype(str).map(state_to_place_map)
pers_mig['last_mx_loc'] = pers_mig['dostatel'].astype(str) + "-" + pers_mig['doplacel'].astype(str)
pers_mig['last_mx_loc'] = pers_mig['last_mx_loc'].astype(str).map(state_to_place_map)
pers_mig['last_mx_loc'] = pers_mig['last_mx_loc'].fillna("-99")

# If the person never migrated from their place of birth, use that as the origin location, otherwise place of origin is last known place living in MX
pers_mig['origin_loc'] = np.where(pers_mig['last_mx_loc'] == "-99", pers_mig['born_loc'], pers_mig['last_mx_loc'])

# pers_mig['us_loc'] = pers_mig['usstate1'].astype(str) + "-" + pers_mig['usplace1'].astype(str)
pers_mig['us_loc'] = pers_mig['usplace1'].astype(str).map(us_places)
pers_mig['us_loc'] = pers_mig['us_loc'].str.replace("PMSA", "")
pers_mig['us_loc'] = pers_mig['us_loc'].str.replace("MSA", "")
pers_mig.head(5)

,placebrn,statebrn,dostatel,doplacel,usyr1,usdur1,usdoc1,usstate1,usplace1,usmar1,...,crscoy1,crscst1,crspay1,crsdep1,crsyes1,cross_loc,born_loc,last_mx_loc,origin_loc,us_loc
0,0,11,8888,8888,1972,12,8,114,1600,,...,1,9999.0,9999,1,1,"Tijuana, Baja California del Norte",Guanajuato,-99,Guanajuato,"Chicago, IL"
1,0,11,8888,8888,1980,36,8,105,7320,,...,1,150.0,9999,0,1,"Tijuana, Baja California del Norte",Guanajuato,-99,Guanajuato,"San Diego, CA"
2,0,11,8888,8888,1978,12,8,105,4480,,...,1,75.0,9999,1,1,"Tijuana, Baja California del Norte",Guanajuato,-99,Guanajuato,"Los Angeles-Long Beach, CA"
3,8,14,11,0,1980,6,8,105,7400,,...,2,8888.0,8888,1,1,"Tijuana, Baja California del Norte","Arandas, Jalisco",Guanajuato,Guanajuato,"San Jose, CA"
4,0,11,8888,8888,1975,156,8,114,1600,,...,2,8888.0,8888,0,1,"Nuevo Laredo, Tamaulipas",Guanajuato,-99,Guanajuato,"Chicago, IL"


In [149]:
flows = pers_mig[["crsyr1", "origin_loc", "cross_loc", "us_loc"]]
flows

,crsyr1,origin_loc,cross_loc,us_loc
0,1972,Guanajuato,"Tijuana, Baja California del Norte","Chicago, IL"
1,1980,Guanajuato,"Tijuana, Baja California del Norte","San Diego, CA"
2,1978,Guanajuato,"Tijuana, Baja California del Norte","Los Angeles-Long Beach, CA"
3,1980,Guanajuato,"Tijuana, Baja California del Norte","San Jose, CA"
4,1975,Guanajuato,"Nuevo Laredo, Tamaulipas","Chicago, IL"
...,...,...,...,...
5316,2004,Oaxaca,"Tijuana, Baja California del Norte","Orange County, CA"
5317,2016,Oaxaca,"Tijuana, Baja California del Norte","Orange County, CA"
5318,1985,Oaxaca,"Tijuana, Baja California del Norte","Orange County, CA"
5319,1994,Oaxaca,"Tijuana, Baja California del Norte","Los Angeles-Long Beach, CA"


In [150]:
flows.to_csv("./good_data/hhead_flows.csv", index = False)

In [151]:
from geopy.geocoders import Nominatim
from shapely.geometry import Point, LineString
import geopandas as gpd
import folium 
import matplotlib.pyplot as plt
import random

In [152]:
def get_coordinates(city_list):
    """Takes a list of cities and returns a dictionary of the cities and their corresponding coordinates."""
    geolocator = Nominatim(user_agent="location script")
    dicto = {}
    
    for city in city_list:
        try:
            location = geolocator.geocode(city)
            coordinate_values = (location.longitude, location.latitude)  #in geopandas, the x value corresponds to the longitude while the y value, the latitude(Just in case you were wondering why it was *location.longitude, location.latitude* and not the other way round )
            dicto[city] = coordinate_values #adding the coordinate pair to the dictionary at the end of every loop            
        except Exception as e:
            dicto[city] = "Fail"
            
    return dicto #finally retruns the dict 

In [153]:
cross_coords = get_coordinates(flows['cross_loc'].unique())
print("cross done")
origin_coords = get_coordinates(flows['origin_loc'].unique())
print("origin done")
us_coords = get_coordinates(flows['us_loc'].unique())
print("us done")

cross done
origin done
us done


In [154]:
# flows = pd.read_csv("./good_data/hhead_flows_wcoords.csv")

flows['origin_coords'] = flows['origin_loc'].map(origin_coords)
flows['cross_coords'] = flows['cross_loc'].map(cross_coords)
flows['us_coords'] = flows['us_loc'].str.replace("PMSA", "").str.replace("MSA", "").map(us_coords)
flows

/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

,crsyr1,origin_loc,cross_loc,us_loc,origin_coords,cross_coords,us_coords
0,1972,Guanajuato,"Tijuana, Baja California del Norte","Chicago, IL","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-87.6244212, 41.8755616)"
1,1980,Guanajuato,"Tijuana, Baja California del Norte","San Diego, CA","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-117.1627728, 32.7174202)"
2,1978,Guanajuato,"Tijuana, Baja California del Norte","Los Angeles-Long Beach, CA","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-118.2400932, 33.7689643)"
3,1980,Guanajuato,"Tijuana, Baja California del Norte","San Jose, CA","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-121.890583, 37.3361905)"
4,1975,Guanajuato,"Nuevo Laredo, Tamaulipas","Chicago, IL","(-101.0, 20.9876996)","(-99.5185992, 27.4761506)","(-87.6244212, 41.8755616)"
...,...,...,...,...,...,...,...
5316,2004,Oaxaca,"Tijuana, Baja California del Norte","Orange County, CA","(-96.5, 17.0)","(-116.964662, 32.5010188)","(-117.8704931, 33.7500378)"
5317,2016,Oaxaca,"Tijuana, Baja California del Norte","Orange County, CA","(-96.5, 17.0)","(-116.964662, 32.5010188)","(-117.8704931, 33.7500378)"
5318,1985,Oaxaca,"Tijuana, Baja California del Norte","Orange County, CA","(-96.5, 17.0)","(-116.964662, 32.5010188)","(-117.8704931, 33.7500378)"
5319,1994,Oaxaca,"Tijuana, Baja California del Norte","Los Angeles-Long Beach, CA","(-96.5, 17.0)","(-116.964662, 32.5010188)","(-118.2400932, 33.7689643)"


In [155]:
flows.to_csv("./good_data/hhead_flows_wcoords_v4.csv", index = False)